# Libraries

In [ ]:
import warnings
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt

from torchsom.core import SOM
from torchsom.visualization import SOMVisualizer, VisualizationConfig

from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    root_mean_squared_error,
    r2_score,
)
from sklearn.exceptions import ConvergenceWarning, DataConversionWarning

warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=DataConversionWarning)

In [ ]:
random_seed = 42
torch.manual_seed(random_seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Preprocessing 

In [ ]:
energy_df = pd.read_csv(
    filepath_or_buffer="../data/notebooks/energy_efficiency.csv",
)

In [ ]:
# energy_df_scaled = energy_df
scaler = StandardScaler()
energy_df_scaled = pd.DataFrame(scaler.fit_transform(energy_df), columns=energy_df.columns)

In [ ]:
energy_df_scaled.head()

In [ ]:
energy_df_scaled.describe()

In [ ]:
feature_names = energy_df_scaled.columns.to_list()[:-2]
feature_names

In [ ]:
energy_df_scaled.shape

In [ ]:
"""
1. Create a tensor from the energy df and separate the features and the target
2. Randomly shuffle the data
3. Split the data into training and testing sets
"""
energy_torch = torch.tensor(energy_df_scaled.to_numpy(dtype=np.float32), device=device)
all_features = energy_torch[:, :-2]
all_targets_heating, all_targets_cooling = energy_torch[:, -2], energy_torch[:, -1]

shuffled_indices = torch.randperm(len(all_features), device=device)
all_features = all_features[shuffled_indices]
all_targets_heating, all_targets_cooling = all_targets_heating[shuffled_indices], all_targets_cooling[shuffled_indices]

train_ratio = 0.8
train_count = int(train_ratio * len(all_features))

train_features = all_features[:train_count]
train_targets_heating, train_targets_cooling = all_targets_heating[:train_count], all_targets_cooling[:train_count]

test_features = all_features[train_count:]
test_targets_heating, test_targets_cooling = all_targets_heating[train_count:], all_targets_cooling[train_count:]

print(train_features.shape, test_features.shape)
print(train_targets_heating.shape, train_targets_cooling.shape, test_targets_heating.shape, test_targets_cooling.shape)

# TorchSOM

In [ ]:
som = SOM(
    x=35,
    y=20,
    sigma=2.5,
    learning_rate=0.95,
    neighborhood_order=3,
    epochs=125,
    batch_size=16,
    topology="rectangular",
    distance_function="euclidean",
    neighborhood_function="gaussian",
    num_features=all_features.shape[1],
    lr_decay_function="asymptotic_decay",
    sigma_decay_function="asymptotic_decay",
    initialization_mode="pca",
    device=device,
    random_seed=random_seed,
)

In [ ]:
som.initialize_weights(
    data=train_features,
    mode=som.initialization_mode
)

In [ ]:
QE, TE = som.fit(
    data=train_features
)

In [ ]:
bmus_map = som.build_map(
    "bmus_data",
    data=train_features,
    # target=train_targets,
    return_indices=True,
    batch_size=train_features.shape[0],
)

In [ ]:
visualizer = SOMVisualizer(som=som, config=VisualizationConfig(save_format="pdf"))
save_path = f"results/energy/{som.topology}" # Set to None if you want a direct plot

In [ ]:
visualizer.plot_training_errors(
    quantization_errors=QE, 
    topographic_errors=TE, 
    save_path=save_path
)

In [ ]:
visualizer.plot_distance_map(
    # fig_name="distance_map",
    save_path=save_path,
    distance_metric=som.distance_fn_name,
    neighborhood_order=som.neighborhood_order,
    scaling="sum",
)

In [ ]:
visualizer.plot_hit_map(
    # fig_name="hit_map",
    data=train_features,
    save_path=save_path,
    batch_size=train_features.shape[0],
)

In [ ]:
visualizer.plot_component_planes(
    component_names=feature_names,
    save_path=save_path
)

### Heating Target

In [ ]:
heating_path = save_path + "/heating"

In [ ]:
visualizer.plot_metric_map(
    # fig_name="mean_metric_map",
    data=train_features,
    target=train_targets_heating,
    reduction_parameter="mean",
    save_path=heating_path,
    bmus_data_map=bmus_map,
)

In [ ]:
visualizer.plot_metric_map(
    # fig_name="std_metric_map",
    data=train_features,
    target=train_targets_heating,
    reduction_parameter="std",
    save_path=heating_path,
    bmus_data_map=bmus_map,
)

In [ ]:
visualizer.plot_rank_map(
    # fig_name="rank_map",
    bmus_data_map=bmus_map,
    target=train_targets_heating,
    save_path=heating_path
)

In [ ]:
visualizer.plot_score_map(
    # fig_name="score_map",
    bmus_data_map=bmus_map,
    target=train_targets_heating,
    total_samples=train_features.shape[0],
    save_path=heating_path,
)

### Cooling Target

In [ ]:
cooling_path = save_path + "/cooling"

In [ ]:
visualizer.plot_metric_map(
    # fig_name="mean_metric_map",
    data=train_features,
    target=train_targets_cooling,
    reduction_parameter="mean",
    save_path=cooling_path,
    bmus_data_map=bmus_map,
)

In [ ]:
visualizer.plot_metric_map(
    # fig_name="std_metric_map",
    data=train_features,
    target=train_targets_cooling,
    reduction_parameter="std",
    save_path=cooling_path,
    bmus_data_map=bmus_map,
)

In [ ]:
visualizer.plot_rank_map(
    # fig_name="rank_map",
    bmus_data_map=bmus_map,
    target=train_targets_cooling,
    save_path=cooling_path
)

In [ ]:
visualizer.plot_score_map(
    # fig_name="score_map",
    bmus_data_map=bmus_map,
    target=train_targets_cooling,
    total_samples=train_features.shape[0],
    save_path=cooling_path,
)

# Prediction of the Heating Target
Here, we do not add the testing samples in the SOM BMUs map.  
In forecasting or process control, it is interesting to add overtime the new elements in the SOM and potentially to update/refit it with a certain frequency.

In [ ]:
predictions = []
for idx, (test_feature, test_target) in enumerate(zip(test_features, test_targets_heating)):
        
    collected_features, collected_targets = som.collect_samples(
        query_sample=test_feature,
        historical_samples=train_features,
        historical_outputs=train_targets_heating,
        min_buffer_threshold=150, # Collect 425 historical samples to train a model
        bmus_idx_map=bmus_map,
    )
    
    X = collected_features.cpu().numpy()
    y = collected_targets.cpu().numpy().ravel()
    test_feature_np = test_feature.cpu().numpy().reshape(1, -1)  
    
    reg = MLPRegressor(
        hidden_layer_sizes=(8, 16, 16, 8),
        max_iter=200,
        learning_rate_init=0.008,
        activation="relu",
        solver="adam",
        batch_size='auto', 
        random_state=random_seed,
        shuffle=True,
        verbose=False,
    ).fit(X, y)
    
    # plt.plot(reg.loss_curve_)
    # plt.xlabel("Iteration")
    # plt.ylabel("Loss")
    # plt.title("MLPRegressor Training Loss Curve")
    # plt.grid(True)
    # plt.show()
    
    reg_prediction = reg.predict(test_feature_np)
    predictions.append(reg_prediction[0]) 

In [ ]:
y_pred = np.array(predictions)
y_true = test_targets_heating.cpu().numpy()     

mae = mean_absolute_error(y_true, y_pred) 
mse = mean_squared_error(y_true, y_pred)
rmse = root_mean_squared_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)

print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R2: {r2:.2f}")

In [ ]:
min_val = min(min(y_pred), min(y_true))
max_val = max(max(y_pred), max(y_true))

plt.figure(figsize=(10, 5))
plt.scatter(
    y_true, 
    y_pred, 
    alpha=0.8,
    color="blue",
    marker="o",
    edgecolor="black",
    s=50,
    label="Predictions",
)
plt.plot(
    [min_val, max_val],
    [min_val, max_val],
    label="Perfect Prediction",
    color="red",
    alpha=0.8,
    linewidth=2
)
plt.xlabel("Ground Truth Values - Heating")
plt.ylabel("Predictions")
plt.show()